<a href="https://colab.research.google.com/github/nelson-ewert/calculadora-aluguel/blob/master/calculadora_de_imoveis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import numpy as np
import pandas as pd
from time import sleep

In [ ]:
def coleta_dados(url, zona):
  conteudo = requests.get(url).content
  sopa = BeautifulSoup(conteudo)
  lista_zonas = []
  lista_preços = []
  preços = sopa.find_all('span', class_='price-tag-fraction')
  for elemento in preços:
    p = re.findall('<span class="price-tag-fraction">(.*)</span>', str(elemento))[0]
    p = float(p) * 1000
    lista_preços.append(p)
    lista_zonas.append(zona)

  sopa2 = BeautifulSoup(conteudo)
  area_e_quartos = sopa2.find_all('ul', class_="ui-search-card-attributes ui-search-item__group__element")
  lista_elementos = []
  for elemento in area_e_quartos:
    lista_elementos.append(elemento.text)
  re_quartos = '(.) quarto|quartos'
  re_area = '(.*)m²'
  lista_quartos = []
  lista_area = []
  for el in lista_elementos:
    quarto = re.findall(re_quartos, str(el))
    if quarto != []:
      lista_quartos.append(float(quarto[0]))
    else:
      lista_quartos.append(np.nan)
    area = re.findall(re_area, str(el))
    if area != []:
      area[0] = area[0].replace(",",".")
      lista_area.append(float(area[0]))
    else:
      lista_area.append(np.nan)

  df = pd.DataFrame({'preço': lista_preços, 'quartos': lista_quartos, 'area': lista_area, 'zona': lista_zonas})  
  return df

In [ ]:
paginas = ['', '_Desde_49', '_Desde_97', '_Desde_145']
zonas = ['norte', 'sul', 'leste', 'oeste']

dataframes = []

for zona in zonas:
  for pagina in paginas:
    url = f'https://imoveis.mercadolivre.com.br/apartamentos/aluguel/sao-paulo/sao-paulo-zona-{zona}/{pagina}'
    print("Coletando dados de: ", url)
    df = coleta_dados(url, zona)
    dataframes.append(df)
    sleep(2)

Coletando dados de:  https://imoveis.mercadolivre.com.br/apartamentos/aluguel/sao-paulo/sao-paulo-zona-norte/
Coletando dados de:  https://imoveis.mercadolivre.com.br/apartamentos/aluguel/sao-paulo/sao-paulo-zona-norte/_Desde_49
Coletando dados de:  https://imoveis.mercadolivre.com.br/apartamentos/aluguel/sao-paulo/sao-paulo-zona-norte/_Desde_97
Coletando dados de:  https://imoveis.mercadolivre.com.br/apartamentos/aluguel/sao-paulo/sao-paulo-zona-norte/_Desde_145
Coletando dados de:  https://imoveis.mercadolivre.com.br/apartamentos/aluguel/sao-paulo/sao-paulo-zona-sul/
Coletando dados de:  https://imoveis.mercadolivre.com.br/apartamentos/aluguel/sao-paulo/sao-paulo-zona-sul/_Desde_49
Coletando dados de:  https://imoveis.mercadolivre.com.br/apartamentos/aluguel/sao-paulo/sao-paulo-zona-sul/_Desde_97
Coletando dados de:  https://imoveis.mercadolivre.com.br/apartamentos/aluguel/sao-paulo/sao-paulo-zona-sul/_Desde_145
Coletando dados de:  https://imoveis.mercadolivre.com.br/apartamentos/al

In [ ]:
df_ML = pd.concat(dataframes, ignore_index=True)

In [ ]:
df_ML.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   preço    768 non-null    float64
 1   quartos  767 non-null    float64
 2   area     767 non-null    float64
 3   zona     768 non-null    object 
dtypes: float64(3), object(1)
memory usage: 24.1+ KB
